# Algorithmic Economics - HW 3
#### Kacper Szczepański, Jakub Józefowicz

Let's install the required libraries first

In [1]:
! pip install numpy seaborn pandas

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


We define the player and battlefield representation:

In [2]:
from dataclasses import dataclass
from typing import List
import numpy as np

# battlefield can just be a list of ints

@dataclass
class Player:
    num_resources: int # How many resources the player has available for allocation
    strategy: List[float] # The players mixed strategy that will evolve

The best response functions, which although similar are not exactly symmetrical because of how draws are handled.

In [3]:
def bestPureResponseAtt(att_player, def_player, battlefields):
    num_resources = att_player.num_resources
    def_strat = def_player.strategy
    for v in def_strat:
        assert v >= 0.0 and v <= 1.0
    field_val = []
    for i, (prob, util) in enumerate(zip(def_strat, battlefields, strict=True)):
        field_val.append(((1 - prob) * util, i))
    field_val = sorted(field_val)[-num_resources:]
    res = [0.0] * len(def_strat)
    for _, i in field_val:
        res[i] = 1.0
    return res


def bestPureResponseDef(att_player, def_player, battlefields):
    num_resources = def_player.num_resources
    att_strat = att_player.strategy
    for v in att_strat:
        assert v >= 0.0 and v <= 1.0
    field_val = []
    for i, (prob, util) in enumerate(zip(att_strat, battlefields, strict=True)):
        field_val.append((prob * util, i))
    field_val = sorted(field_val)[-num_resources:]
    res = [0.0] * len(att_strat)
    for _, i in field_val:
        res[i] = 1.0
    return res

We also have to implement a calculation of $\epsilon$, this is bounded from the bottom by the difference in paysoff between the players when reacting with any of their best responses against the opponent's mixed strategy.

Let $S$ be the set of pure strategies available to a given player, then:
$$
todo
$$

5 3 4
2 2 2 2 0

In [4]:
def getEpsilon(
    att_player, def_player, battlefields, best_resp_att=None, best_resp_def=None
):
    att_strat = att_player.strategy
    def_strat = def_player.strategy
    if best_resp_att is None:
        best_resp_att = bestPureResponseAtt(att_player, def_player, battlefields)
    if best_resp_def is None:
        best_resp_def = bestPureResponseDef(att_player, def_player, battlefields)
    pay_att = 0.0
    pay_def = 0.0
    for i in range(n):
        pay_att += best_resp_att[i] * battlefields[i] * (1 - def_strat[i])
        pay_def += (1 - best_resp_def[i]) * battlefields[i] * att_strat[i]

    return abs(pay_att - pay_def)

We are now able to implement ficticious play.

In [5]:
def ficticiousPlay(  # todo intial conditions player
    battlefields, num_res_att, num_res_def, epsilon=0.001, max_iters=1_000_000
):
    assert num_res_att > 0 and num_res_att < len(battlefields)
    assert num_res_def > 0 and num_res_def < len(battlefields)
    assert num_res_att < num_res_def
    num_battlefields = len(battlefields)
    att_play = Player(num_res_att, [0.0] * num_battlefields)
    def_play = Player(num_res_def, [0.0] * num_battlefields)
    epsilons = []
    for t in range(1, max_iters + 1):
        resp_att = bestPureResponseAtt(att_play, def_play, battlefields)
        resp_def = bestPureResponseDef(att_play, def_play, battlefields)
        err = getEpsilon(att_play, def_play, battlefields, resp_att, resp_def)
        epsilons.append(err)
        if err <= epsilon:
            break
        for i, (cur, new) in enumerate(zip(att_play.strategy, resp_att)):
            att_play.strategy[i] = (cur * (t - 1) + new) / t
        for i, (cur, new) in enumerate(zip(def_play.strategy, resp_def)):
            def_play.strategy[i] = (cur * (t - 1) + new) / t

    return att_play.strategy, def_play.strategy, epsilons # len(epsilons) gives us the number of iterations the algorithm ran for

Let's generate some nice variety for the input analysis, we will consider games with 10, 100, 1000 and 10000 battlefields for each we will check situations where the attacker and defender have a low(< ~15%), medium (~40-60%) or high number(>~85%) or resources in comparison to the number of battlefields

In [13]:
import random as r
import pandas as pd
import math

r.seed(42)
input_ranges = ['low', 'mid', 'high']
range_divisors = [(10000, 1/0.15), (1/0.4, 1/0.6), (1/0.85, 1)]
field_sizes = [10,100,1000,10000]
data = []
column_names = ['field_size', 'tokens_att', 'tokens_def', 'range_att', 'range_def']

for field_size in field_sizes:
    for att_range in range(len(input_ranges)):
        for def_range in range(att_range, len(input_ranges)):
            for num_inputs in range(100):
                min_att = math.ceil(field_size/range_divisors[att_range][0])
                min_att = min(field_size - 2, min_att)
                max_att = math.floor(field_size/range_divisors[att_range][1])
                max_att = max(min_att+1, max_att)
                max_att = min(max_att, field_size-1)
                assert(min_att < max_att)
                tokens_att = r.randrange(min_att, max_att)
                min_def = math.ceil(field_size/range_divisors[def_range][0])
                min_def = max(tokens_att+1, min_def)
                max_def = math.floor(field_size/range_divisors[att_range][1])
                max_def = max(min_def+1, max_def)
                tokens_def = r.randrange(min_def, max_def)
                assert(tokens_att < tokens_def)
                assert(tokens_def < field_size)
                data.append((field_size, tokens_att, tokens_def, input_ranges[att_range], input_ranges[def_range]))

epsilon_histories



AssertionError: 